In [2]:
import psycopg2 as ps
import pandas as pd
import time

In [3]:
df = pd.read_csv('youtube_api_dataframe_2nd_pull.csv', index_col=0)
df.head()

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,Z3jG0dpeAw0,2v2 WITH THEVIPER! | AoE4,2021-12-10,1816,94,2,18
1,c-cKtv_zczU,The Unfinished CLASH ft. TheViper | AoE4 | Grubby,2021-12-09,5739,193,1,17
2,KdO_zN96bis,My MAP TIERLIST! | AoE4,2021-12-09,4068,128,1,40
3,Mz5y7Adwzi4,COLLAB ft. TheViper! - AoE4 Balance Discussion...,2021-12-08,12214,460,6,83
4,msxpkiL_DsU,Testing FIRELANCERS! | AoE4 | Grubby,2021-12-07,5657,140,3,29


In [12]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected')
        return conn

In [13]:
host_name = 'xxxx'
dbname = 'xxxx'
port = 'xxxx'
username = 'xxxx'
password = 'xxxx'
conn = None

conn = connect_to_db(host_name,dbname,port,username,password)

Connected


In [6]:
#create table

def create_table(curr):
    create_table_command = (''' CREATE TABLE IF NOT EXISTS videos3 (
                                video_id VARCHAR(255),
                                video_title TEXT NOT NULL,
                                upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                                view_count INTEGER NOT NULL,
                                like_count INTEGER NOT NULL,
                                dislike_count INTEGER NOT NULL,
                                comment_count INTEGER NOT NULL
                            )''')
    
    curr.execute(create_table_command)
    
##########################
    

#check if video exists
def check_if_video_exists(curr,video_id):
    query = (''' SELECT video_id FROM videos3 WHERE video_id = %s''') 
    curr.execute(query, (video_id,))

    return curr.fetchone() is not None  


######################



#update row if video exists
def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):

    query = (''' UPDATE videos3
                SET video_title = %s,
                    view_count = %s,
                    like_count = %s,
                    dislike_count = %s,
                    comment_count = %s
                WHERE video_id = %s;''')

    vars_to_update = (video_title, view_count, like_count, dislike_count, comment_count, video_id)
    curr.execute(query,vars_to_update)

    
######################
    
    
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'view_count', 'like_count',
                                  'dislike_count', 'comment_count'])

    for i,row in df.iterrows(): 
        if check_if_video_exists(curr, row['video_id']): 
            update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])
        else: 
            tmp_df = tmp_df.append(row)
        
    return tmp_df



#######################
        
        
#insert command

def insert_into_table(curr,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count):

    insert_into_videos = ("""INSERT INTO videos3 (video_id, video_title, upload_date,
                        view_count, like_count, dislike_count,comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, dislike_count, comment_count)
    
    
    curr.execute(insert_into_videos, row_to_insert) 
        
#############################


        
#insert videos into the db
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['dislike_count'], row['comment_count'])       

In [7]:
curr = conn.cursor()



In [8]:
create_table(curr)
conn.commit()


In [9]:
#first pull
#new_video_df = update_db(curr,df)
#conn.commit()



#2nd pull
new_video_df2 = update_db(curr,df)
conn.commit()

In [10]:
#first pull
#append_from_df_to_db(curr,new_video_df)
#conn.commit()



#2nd pull
append_from_df_to_db(curr,new_video_df2)
conn.commit()

In [11]:
df

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,Z3jG0dpeAw0,2v2 WITH THEVIPER! | AoE4,2021-12-10,1816,94,2,18
1,c-cKtv_zczU,The Unfinished CLASH ft. TheViper | AoE4 | Grubby,2021-12-09,5739,193,1,17
2,KdO_zN96bis,My MAP TIERLIST! | AoE4,2021-12-09,4068,128,1,40
3,Mz5y7Adwzi4,COLLAB ft. TheViper! - AoE4 Balance Discussion...,2021-12-08,12214,460,6,83
4,msxpkiL_DsU,Testing FIRELANCERS! | AoE4 | Grubby,2021-12-07,5657,140,3,29
...,...,...,...,...,...,...,...
515,de-aCP1CXgc,Grubby | Warcraft 3 REFORGED - QA With Classic...,2018-11-03,76908,1643,30,430
516,IYHBqNnHXQ4,Grubby | Warcraft 3 TFT | 1.30 | ORC v NE on T...,2018-10-30,53466,765,19,55
517,3HGF9UghHAc,Grubby | Warcraft 3 TFT | 1.30 | HU v UD on Tw...,2018-10-30,38951,527,15,55
518,F-Wzhr7HXQQ,Grubby | Warcraft 3 TFT | 1.30 | ORC v UD on C...,2018-10-28,29752,686,15,70
